<a href="https://colab.research.google.com/github/KRINA05/krina/blob/master/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Jun 26 07:45:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': Operation not supported
 2019-11-30-Stats-Parikshit-A-Advani-HM-1909.ipynb
'4.25 mathematics.pdf'
'4.60 Statistics.pdf'
 6208
 CANCER.gform
'Case control study of breast cancer and its relevance with lung cancer and their risk factors.gdoc'
'Colab Notebooks'
'Copy of Case control study of breast cancer and its relevance with lung cancer and their risk factors.gdoc'
'Copy of Helmet (100) (1).jpg'
'Copy of Helmet (100).jpg'
'Copy of Helmet (101) (1).jpg'
'Copy of Helmet (10) (1).jpg'
'Copy of Helmet (101).jpg'
'Copy of Helmet (10) (1).png'
'Copy of Helmet (102) (1).jpg'
'Copy of Helmet (102).jpg'
'Copy of Helmet (103) (1).jpg'
'Copy of Helmet (103).jpg'
'Copy of Helmet (104) (1).jpg'
'Copy of Helmet (104).jpg'
'Copy of Helmet (105) (1).jpg'
'Copy of Helmet (105).jpg'
'Copy of Helmet (106) (1).jpg'
'Copy 

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 13772 (delta 14), reused 30 (delta 14), pack-reused 13738
Receiving objects: 100% (13772/13772), 12.36 MiB | 1.84 MiB/s, done.
Resolving deltas: 100% (9390/9390), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variabl

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "helmet" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-06-26 07:47:49--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   417KB/s    in 6m 31s  

2020-06-26 07:54:23 (405 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/2Q__ (1).jpg   
  inflating: data/obj/2Q__ (1).txt   
  inflating: data/obj/2Q__ (3).jpg   
  inflating: data/obj/2Q__ (3).txt   
  inflating: data/obj/2Q__ (4).jpg   
  inflating: data/obj/2Q__ (4).txt   
  inflating: data/obj/9k_ (1).jpg    
  inflating: data/obj/9k_ (1).txt    
  inflating: data/obj/9k_ (2).jpg    
  inflating: data/obj/9k_ (2).txt    
  inflating: data/obj/9k_ (4).jpg    
  inflating: data/obj/9k_ (4).txt    
  inflating: data/obj/9k_ (6).jpg    
  inflating: data/obj/9k_ (6).txt    
  inflating: data/obj/9k_ (8).jpg    
  inflating: data/obj/9k_ (8).txt    
  inflating: data/obj/9k_.jpg        
  inflating: data/obj/9k_.txt        
  inflating: data/obj/BikesHelmets1.png  
  inflating: data/obj/BikesHelmets1.txt  
  inflating: data/obj/BikesHelmets100.png  
  inflating: data/obj/BikesHelmets100.txt  
  inflating: data/obj/BikesHelmets109.png  
  inflating: data/obj/BikesHelmets109.txt  
  inflating: data/o

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Streaming output truncated to the last 5000 lines.

['15', '0.583333', '0.427083', '0.576190', '0.595833']
250 data/obj/Helmet (404).txt
0 0.583333 0.427083 0.576190 0.595833
15 0.863469 0.209677 0.191882 0.397849

['15', '0.863469', '0.209677', '0.191882', '0.397849']
251 data/obj/images - 2020-06-24T183918.717.txt
0 0.863469 0.209677 0.191882 0.397849
15 0.436364 0.393443 0.080000 0.109290

['15', '0.436364', '0.393443', '0.080000', '0.109290']
252 data/obj/images - 2020-06-24T181525.606.txt
0 0.436364 0.393443 0.080000 0.109290
15 0.666667 0.210526 0.214689 0.175439

['15', '0.666667', '0.210526', '0.214689', '0.175439']
253 data/obj/Helmet (737).txt
0 0.666667 0.210526 0.214689 0.175439
15 0.393750 0.161765 0.192500 0.315126

['15', '0.393750', '0.161765', '0.192500', '0.315126']
254 data/obj/BikesHelmets476.txt
0 0.393750 0.161765 0.192500 0.315126
15 0.532075 0.171053 0.128302 0.194737

['15', '0.532075', '0.171053', '0.128302', '0.194737']
255 data/obj/Helmet (26).txt
0 0.532075

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/images - 2020-06-24T184633.364.jpg', 'data/obj/images - 2020-06-24T184343.141.jpg', 'data/obj/Helmet (232).jpg', 'data/obj/images - 2020-06-24T181114.415.jpg', 'data/obj/Helmet (764).jpg', 'data/obj/Helmet (611).jpg', 'data/obj/Helmet (287).jpg', 'data/obj/Helmet (133).jpg', 'data/obj/images - 2020-06-24T181429.702.jpg', 'data/obj/images - 2020-06-24T181502.443.jpg', 'data/obj/images - 2020-06-24T181339.518.jpg', 'data/obj/images - 2020-06-24T181240.663.jpg', 'data/obj/images - 2020-06-24T180037.666.jpg', 'data/obj/images - 2020-06-24T181405.158.jpg', 'data/obj/images - 2020-06-24T181513.195.jpg', 'data/obj/images - 2020-06-24T181504.201.jpg', 'data/obj/Helmet (330).jpg', 'data/obj/Helmet (871).jpg', 'data/obj/Helmet (472).jpg', 'data/obj/images - 2020-06-24T181507.658.jpg', 'data/obj/Helmet (631).jpg', 'data/obj/Helmet (867).jpg', 'data/obj/images - 2020-06-24T184040.594.jpg', 'data/obj/images - 2020-06-24T181121.672.jpg', 'data/obj/images (100).jpg', 'data/obj/Helmet (498)

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 138206, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.869562, GIOU: 0.864866), Class: 0.998453, Obj: 0.913188, No Obj: 0.000627, .5R: 1.000000, .75R: 1.000000, count: 1, class_loss = 0.002073, iou_loss = 0.010040, total_loss = 0.012112 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.787904, GIOU: 0.783887), Class: 0.999453, Obj: 0.798249, No Obj: 0.000524, .5R: 1.000000, .75R: 0.666667, count: 3, class_loss = 0.047208, iou_loss = 0.129291, total_loss = 0.176499 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000001, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 138210, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: